In [ ]:
import pandas as pd
import numpy as np
from src.api_osmr import get_matrix, parse_coordinates
from src.routing import optimize_routes, plot_route
import folium

# Load data
alpa = pd.read_csv("data/input/Data_Alfamart Indomaret_South Jakarta.csv")

# Rename columns
alpa.rename(columns={'longitude': 'lng'}, inplace=True)
alpa.rename(columns={'latitude': 'lat'}, inplace=True)

# Group by kecamatan and calculate median
district = alpa.groupby(['nama_kecamatan'])[['lat','lng']].median().reset_index()

# Extract the list of districts
list_camat = district.nama_kecamatan

# Parse coordinates for the districts
koordinat = parse_coordinates(district)

# Get distance matrix using the OSRM API
respon_jarak = get_matrix(koordinat)

# Convert distance to integer values (required by ortools)
jarak = np.array(respon_jarak["distances"]).astype(int)

# Set starting district for the route
depot_district = "Tebet"

# Find index of the starting district
index_depot = int(list_camat.index[list_camat == depot_district][0])

# Optimize route
rute_idx = optimize_routes(jarak, index_depot, list(list_camat))

# Create a DataFrame with the best route
alpa_best_rute = district.iloc[rute_idx]

# Get coordinates of the best route
koordinat = list(zip(alpa_best_rute["lat"], alpa_best_rute["lng"]))

# Plot the best route on a Folium map
map_ku = plot_route(koordinat, dist=30000, zoom_start=11)

# Calculate total distance of all routes
total_distance = np.sum(jarak[rute_idx[:-1], rute_idx[1:]])

# Plot depot
depot_coords = (alpa_best_rute.iloc[0]['lat'], alpa_best_rute.iloc[0]['lng'])
folium.Marker(location=depot_coords, icon=folium.Icon(color='red', icon='home', prefix='fa'), tooltip=f"Depot {depot_district}").add_to(map_ku)

# Display the map
map_ku


In [ ]:
pip install ortools
pip install scikit-learn
import pandas as pd
import numpy as np
from src.api_osmr import get_matrix, parse_coordinates
from src.routing import optimize_routes, plot_route
alpa = pd.read_csv("data/input/Data_Alfamart Indomaret_South Jakarta.csv")
alpa.rename(columns={'longitude': 'lng'}, inplace=True)
alpa.rename(columns={'latitude': 'lat'}, inplace=True)
district = alpa.groupby(['nama_kecamatan'])[['lat','lng']].median().reset_index()
# Extract the list of district from the "district" DataFrame
list_camat = district.nama_kecamatan

# Parse koordinat for the district
koordinat = parse_coordinates(district)

# Get the distance matrix for the district using the OSRM API
respon_jarak = get_matrix(koordinat)

# Convert the jarak to integer values (required by ortools)
jarak = np.array(respon_jarak["distances"]).astype(int)

# Set the starting nama_kecamatan for the route (change if needed)
camat_start = "Tebet"

# Find the index of the starting nama_kecamatan in the list_camat
index_depot = int(list_camat.index[list_camat == camat_start][0])

# Optimize the route using the distance matrix and starting nama_kecamatan index
rute_idx = optimize_routes(jarak, index_depot, list(list_camat))
# Create a DataFrame with the best route
alpa_best_rute = district.iloc[rute_idx]

# Get the koordinat of the best route
koordinat = list(zip(alpa_best_rute["lat"], alpa_best_rute["lng"]))

# Plot the best route on a Folium map
map_ku = plot_route(koordinat, dist=30000, zoom_start=11)
map_ku


